In [9]:
#importing libraries
import pandas as pd
import json
import requests
from openpyxl import load_workbook


In [2]:
nyc_geo = pd.read_csv('nyc_geo_dataframe.csv')
nyc_geo.head()

,Unnamed: 0,Neighborhood,Borough,lat,long
0,0,Wakefield,Bronx,40.8947,-73.8472
1,1,Co-op City,Bronx,40.8743,-73.8299
2,2,Eastchester,Bronx,40.8876,-73.8278
3,3,Fieldston,Bronx,40.8954,-73.9056
4,4,Riverdale,Bronx,40.8908,-73.9126


In [25]:
#setting API url, headers, parameters
url = 'https://api.foursquare.com/v3/places/search'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3kLBdN2POhudYIhpCkJ56NK7j2eBAU/KvfmS65pCAXTM='    
}

parameters = {'fields' : 'fsq_id,location,categories,price,rating,popularity,stats',
            'sort' : 'RATING',
            'limit' : '50',
            'near' : ''
           }


In [24]:
#write in excel
book = load_workbook('FoursquareResults.xlsx')
writer = pd.ExcelWriter('FoursquareResults.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)


In [26]:
#create for loop to get all the neighbourhoods
for i in range (0, 306):
    
    #adjusting parameter for each neighborhood
    location = nyc_geo['Neighborhood'].iloc[i] + ', ' + nyc_geo['Borough'].iloc[i]
    parameters['near'] = location
    
    #executing API
    response = requests.get(url, headers=headers, params = parameters)

    response  = response.json()
    df = pd.DataFrame(pd.json_normalize(response, record_path = "results"))
    
    #saving response as csv
    startrow = writer.sheets['Sheet1'].max_row
    
    for sheetname in writer.sheets:
        df.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)

    writer.save()